# Model Context Protocol (MCP) Server Example 
This Jupyter notebook runs on Colab and demonstrates a MCP server running Mellea. 

## Install Ollama and MCP

Before we get started with Mellea, we download, install and serve ollama, and install mcp. We define set_css to wrap Colab output.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &
!uv pip install mcp -q

from IPython.display import HTML, display


def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))


get_ipython().events.register("pre_run_cell", set_css)

## Install Mellea
We run `uv pip install mellea` to install Mellea.

In [ ]:
!uv pip install mellea -q

## Import Mellea and MCP Server

In [ ]:
from mcp.server.fastmcp import FastMCP

from mellea import MelleaSession
from mellea.backends import model_ids
from mellea.backends.ollama import OllamaModelBackend
from mellea.core import ModelOutputThunk, Requirement
from mellea.stdlib.requirements import simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

## Create an MCP Server
Run MCP debug UI with: `uv run mcp dev docs/examples/mcp/mcp_example.py`. We can wrap a simple mcp server around a program and use the server as-is. 

In [ ]:
mcp = FastMCP("Demo")


@mcp.tool()
def write_a_poem(word_limit: int) -> str:
    """Write a poem with a word limit."""
    m = MelleaSession(OllamaModelBackend(model_ids.QWEN3_8B))
    wl_req = Requirement(
        f"Use only {word_limit} words.",
        validation_fn=simple_validate(lambda x: len(x.split(" ")) < word_limit),
    )
    res = m.instruct(
        "Write a poem",
        requirements=[wl_req],
        strategy=RejectionSamplingStrategy(loop_budget=4),
    )
    assert isinstance(res, ModelOutputThunk)
    return str(res.value)


@mcp.resource("greeting://{name}")
def get_greeting(name: str) -> str:
    """Get a personalized greeting."""
    return f"Hello, {name}!"